In [1]:
import os
import time
import shutil
import pickle

import torch
import torch.nn.functional as F

from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
# from tensorboard_logger import configure, log_value

from model import RecurrentAttention
from utils import AverageMeter

from torchvision import transforms, utils, models
from PIL import Image

import torch

import utils
import data_loader

from trainer import Trainer
from config import get_config

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import pandas as pd

import json

In [2]:
def load_inputs(impath):
    to_tens = transforms.ToTensor()
    return to_tens(Image.open(impath).convert('RGB')).unsqueeze(0)

In [3]:
m = open("../../pooling/data/migration_data.json",)
mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
q = 2
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data

,muni_id,num_migrants,class
0,484001001,42055.0,1
1,484001002,4017.0,1
2,484001003,11992.0,1
3,484001004,762.0,1
4,484001005,7551.0,1
...,...,...,...
2326,484032049,2487.0,1
2327,484032050,2024.0,1
2328,484032051,3084.0,1
2329,484032052,2919.0,1


In [4]:
mig_data['class'].value_counts()

0    1166
1    1165
Name: class, dtype: int64

In [5]:
def get_png_names(directory):
    images = []
    for i in os.listdir(directory):
        try:
            if os.path.isdir(os.path.join(directory, i)):
                new_path = os.path.join(directory, i, "pngs")
                image = os.listdir(new_path)[0]
                images.append(os.path.join(directory, i, "pngs", image))
        except:
            pass
    return images


            

image_names = get_png_names("../../pooling/data/MEX/")

y_class, y_mig = [], []

for i in image_names:
        dta = mig_data[mig_data["muni_id"] == i.split("/")[5]]
        if len(dta) != 0:
            y_class.append(dta['class'].values[0])
            y_mig.append(dta['num_migrants'].values[0])

In [6]:
import matplotlib.pyplot as plt
import torchvision

In [7]:
import random

train_num = int(25 * .70)

train_indices = random.sample(range(0, 25), train_num)
val_indices = [i for i in range(0, 25) if i not in train_indices]

In [8]:
import torchvision

batch_size = 1

# train = [(load_inputs(image_paths[i]).squeeze()[:, 0:28, 0:28], ys[i]) for i in range(0, 93)]
# val = [(load_inputs(image_paths[i]).squeeze()[:, 0:28, 0:28], ys[i]) for i in range(93, 133)]

# brighten = torchvision.transforms.functional.adjust_brightness(brightness_factor = 2)


train = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in train_indices]
val = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in val_indices]


train_dl = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = True)

In [9]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  17
Num validation:  8


In [10]:
import numpy as np
from utils import plot_images

import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [11]:
config, unparsed = get_config()

In [12]:
trainer = Trainer(config, (train_dl, val_dl))

In [13]:
import pickle
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from utils import denormalize, bounding_box

def denormalize(T, coords):
    return 0.5 * ((coords + 1.0) * T)

In [40]:
locations_dict = {}

for i in image_namess:
    
    print(i)
    
    muni_id = i.split("/")[5]
    image = load_inputs(i)
    locations = trainer.extract_locations(image)
    
    start = [denormalize(image.shape[2], l) for l in locations]
    start = torch.cat([start[l].unsqueeze(0) for l in range(4)])
        
    B, C, H, W = image.shape
    
    size = int(min(H, W) / 5)
    
    end = start + size
    
    coords_dict = {}
    
    for c in range(0, len(start)):
        
        from_coords = start[c]
        to_coords = end[c]
        
        from_x = from_coords[0][0].item()
        from_y = from_coords[0][1].item()
        
        to_x = to_coords[0][0].item()
        to_y = to_coords[0][1] .item()       
        
        coords_dict['glimpse' + str(c)] = {'from_x': from_x, 'to_x': to_x, 'from_y': from_y, 'to_y': to_y}
    
    locations_dict[i] = coords_dict

../../pooling/data/MEX/484014116/pngs/484014116_2010_1_box484014116_MAY.png
../../pooling/data/MEX/484014060/pngs/484014060_2010_1_box484014060_MAY.png
../../pooling/data/MEX/484014117/pngs/484014117_2010_1_box484014117_MAY.png
../../pooling/data/MEX/484014111/pngs/484014111_2010_1_box484014111_MAY.png
../../pooling/data/MEX/484001001/pngs/484001001_2010_1_box484001001_MAY.png


In [ ]:
# torch.rand(2, 256).flatten(start_dim = 0).unsqueeze(0).shape

In [42]:
with open("sample2.json", "w") as outfile: 
    json.dump(locations_dict, outfile)

In [ ]:
# test = torch.rand(2, 256)
# patch_dict = {}

# for patch in range(0, test.shape[0]):
#     cur_vals = list(test[patch].numpy())
#     patch_dict[patch] = [str(i) for i in cur_vals]
    
# with open("sample.json", "w") as outfile: 
#     json.dump(patch_dict, outfile)